In [1]:
import diagonal_b6 as b6

w = b6.connect_insecure("0.0.0.0:8002")

### A simple example of using the changesets to center the UI at a certain point

Note that this assumes you have, at least, an index containing `edinburgh` in the `./data` folder; perhaps obtained from PlanetOSM:


```
# Download the osm data somewhere
docker run --rm -it -v $PWD:/download openmaptiles/openmaptiles-tools download-osm geofabrik scotland -- -d /download

# Ingest it into an index
b6-ingest-osm --input ../data/scotland-latest.osm.pbf --output data/scotland.index
```

In [11]:
# The Edinburgh Castle; found via OSM.
# https://www.openstreetmap.org/way/4301292#map=18/55.948676/-3.200401
# Note: This has to be a node ( https://www.openstreetmap.org/node/26005891 )
castle = b6.osm_node_id(26005891)

In [3]:
root = b6.FeatureID(b6.FEATURE_TYPE_COLLECTION, "diagonal.works/edinburgh", 0)

In [4]:
w(b6.find_feature(castle))

In [5]:
center    = b6.FeatureID(b6.FEATURE_TYPE_POINT, "diagonal.works/edinburgh/centroid", 0)
new_point = b6.add_point(b6.ll(55.95348352921928, -3.1724975130166477), center, {})

In [6]:
new_collection = b6.add_collection(root, {}, b6.collection(
    # Option 1: Center at some lat/long we pick.
    # b6.pair("centroid", center)

    # Option 2: Center at some NodeID we find from OSM.
    b6.pair("centroid", castle)
))

changes = b6.merge_changes([
    new_point,
    new_collection
])

In [7]:
w(b6.with_change(changes, lambda: b6.changes_to_file("data/edinburgh.yaml")))

'data/edinburgh.yaml'

Now you can restart the world and open the UI like so:

In [8]:
print(f"http://localhost:8001/?r={root}")

http://localhost:8001/?r=/collection/diagonal.works/edinburgh/0
